<a href="https://colab.research.google.com/github/MinhPhanBabsonMSBA/Stock-analysis-/blob/main/Copy_of_LSTM_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:


# Load stock data
ticker = "GS"  # Goldman Sachs example
stock = yf.Ticker(ticker)
df = stock.history(period="1y")  # Fetch last 1 year of data to reduce memory usage

# Use the closing price for prediction
df = df[['Close']]

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df)

# Prepare the dataset for LSTM
def prepare_data(data, look_back=5):  # Reduce look_back to lower memory usage
    X, y = [], []
    for i in range(look_back, len(data)):
        X.append(data[i-look_back:i, 0])
        y.append(data[i, 0])
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X, y

look_back = 5  # Use fewer days to reduce dataset size
X, y = prepare_data(df_scaled, look_back)

# Split data into training and test sets



train_size = int(len(X) * 0.5)  # Reduce training size to 50%
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


# Build LSTM model with fewer units to optimize memory
model = Sequential([
    LSTM(units=10, return_sequences=False, input_shape=(look_back, 1)),
    Dense(units=1)
])
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model with fewer epochs and a smaller batch size
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

# Make predictions
y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1))
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Plot results
plt.figure(figsize=(10,10))
plt.plot(y_test, label='Actual Prices')
plt.plot(y_pred, label='Predicted Prices', linestyle='dashed')
plt.legend()
plt.title(f"LSTM Prediction for {ticker}")
plt.xlabel("Days")
plt.ylabel("Stock Price")
plt.show()




YFRateLimitError: Too Many Requests. Rate limited. Try after a while.

In [ ]:
# Load stock data
ticker = "JPM"  # Goldman Sachs example
stock = yf.Ticker(ticker)
df = stock.history(period="1y")  # Fetch last 1 year of data to reduce memory usage

# Use the closing price for prediction
df = df[['Close']]

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df)

# Prepare the dataset for LSTM
def prepare_data(data, look_back=5):  # Reduce look_back to lower memory usage
    X, y = [], []
    for i in range(look_back, len(data)):
        X.append(data[i-look_back:i, 0])
        y.append(data[i, 0])
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X, y

look_back = 5  # Use fewer days to reduce dataset size
X, y = prepare_data(df_scaled, look_back)

# Split data into training and test sets


train_size = int(len(X) * 0.5)  # Reduce training size to 50%
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


# Build LSTM model with fewer units to optimize memory
model = Sequential([
    LSTM(units=10, return_sequences= False, input_shape=(look_back, 1)),
    Dense(units=1)
])
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model with fewer epochs and a smaller batch size
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

# Make predictions
y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1))
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Plot results
plt.figure(figsize=(10,10))
plt.plot(y_test, label='Actual Prices')
plt.plot(y_pred, label='Predicted Prices', linestyle='dashed')
plt.legend()
plt.title(f"LSTM Prediction for {ticker}")
plt.xlabel("Months")
plt.ylabel("Stock Price")
plt.show()

YFRateLimitError: Too Many Requests. Rate limited. Try after a while.

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load stock data
ticker = "MS"  # Goldman Sachs example
stock = yf.Ticker(ticker)
df = stock.history(period="1y")  # Fetch last 1 year of data to reduce memory usage

# Use the closing price for prediction
df = df[['Close']]

# Scale the data
scaler = MinMaxScaler (feature_range=(0, 1))
df_scaled = scaler.fit_transform(df)

# Prepare the dataset for LSTM
def prepare_data(data, look_back=5):  # Reduce look_back to lower memory usage
    X, y = [], []
    for i in range(look_back, len(data)):
        X.append(data[i-look_back:i, 0])
        y.append(data[i, 0])
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X, y

look_back = 5  # Use fewer days to reduce dataset size
X, y = prepare_data(df_scaled, look_back)

# Split data into training and test sets


train_size = int(len(X) * 0.5)  # Reduce training size to 50%
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


# Build LSTM model with fewer units to optimize memory
model = Sequential([
    LSTM(units=10, return_sequences= False, input_shape=(look_back, 1)),
    Dense(units=1)
])
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model with fewer epochs and a smaller batch size
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

# Make predictions
y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1))
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Plot results
plt.figure(figsize=(10,10))
plt.plot(y_test, label='Actual Prices')
plt.plot(y_pred, label='Predicted Prices', linestyle='dashed')
plt.legend()
plt.title(f"LSTM Prediction for {ticker}")
plt.xlabel("Months")
plt.ylabel("Stock Price")
plt.show()

ERROR:yfinance:$MS: possibly delisted; no price data found  (period=10y)


ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by MinMaxScaler.